In [2]:
# Import necessary libraries
import json
import pandas as pd
import openai
from groq import Groq
from dotenv import load_dotenv
import os
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.auto import tqdm

In [3]:
# Load environment variables
load_dotenv()

True

In [18]:
# Setup project paths
base_folder = 'D:/Projects/datatalks-final-project/'
input_data_folder = base_folder + 'dataset/'


In [19]:
# Get the current working directory (optional step)
os.getcwd()

'd:\\Projects\\datatalks-final-project\\notebooks'

In [20]:
# Setup the OpenAI client to use either Groq, OpenAI.com, or Ollama API
load_dotenv(override=True)
API_HOST = os.getenv("API_HOST")
API_HOST

'groq'

In [21]:
if API_HOST == "groq":
    client = client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
    MODEL_NAME = os.getenv("GROQ_MODEL")

elif API_HOST == "ollama":
    client = openai.OpenAI(
        base_url=os.getenv("OLLAMA_ENDPOINT"),
        api_key="nokeyneeded",
    )
    MODEL_NAME = os.getenv("OLLAMA_MODEL")

elif API_HOST == "openai":
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    MODEL_NAME = os.getenv("OPENAI_MODEL")
else:
    print("No LLM Selected")

In [22]:
client

In [23]:
MODEL_NAME

'llama3-8b-8192'

In [24]:
with open(input_data_folder + 'main_faq_database.json', 'rt') as f_in:
    data = json.load(f_in)

In [27]:
documents = []
for dish in data['dishes']:
    dish_name = dish['dish name']
    for doc in dish['documents']:
        doc['dish_name'] = dish_name  # Add dish_name to each document
        documents.append(doc)

In [28]:
documents[1:3]

[{'id': '142_2',
  'question': 'How many calories does it have?',
  'section': 'calories',
  'text': 'The almond fudge banana cake has 224.8 calories.',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_3',
  'question': 'How much total fat does it contain?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has N/A grams of total fat (PDV).',
  'dish_name': 'almond fudge banana cake'}]

Retreival evaluation

In [29]:
import minsearch

index = minsearch.Index(
    text_fields = ['id', 'question','section','text','dish_name'],
    keyword_fields=['dish_name']
)

index.fit(documents)

In [35]:
#import ground truth dataset
import pandas as pd
ground_df = pd.read_csv(input_data_folder +'gold_standard_data/ground-truth-data.csv')

In [15]:
ground_truth = ground_df.to_dict(orient='records')

In [16]:
documents[1]

{'id': '142_2',
 'question': 'How many calories does it have?',
 'section': 'calories',
 'text': 'The almond fudge banana cake has 224.8 calories.',
 'dish_name': 'almond fudge banana cake'}

In [48]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


In [36]:
def hit_rate2(relevance_total):
    cnt = sum([1 for line in relevance_total if any(line)])
    return cnt / len(relevance_total)


In [49]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [38]:
def mrr2(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank, relevance in enumerate(line):
            if relevance:  # If relevant document is found
                total_score += 1 / (rank + 1)  # rank+1 because rank is 1-based
                break  # Stop after the first relevant document

    return total_score / len(relevance_total)


In [54]:
documents[1]

{'id': '142_2',
 'question': 'How many calories does it have?',
 'section': 'calories',
 'text': 'The almond fudge banana cake has 224.8 calories.',
 'dish_name': 'almond fudge banana cake'}

In [63]:
def minsearch_search(query):
    boost = {

    'question': 2.5,       # High priority: users are searching questions
    'dish_name': 1.7,      # Important for matching specific dishes
    'section': 1.2,        # Moderate boost: helps in filtering by category
    'text': 1.3        # Slight boost: could help match answer content}
    }

    results = index.search(
        query=query,
        filter_dict={'question': query},
        boost_dict=boost,
        num_results=10
    )

    return results

In [44]:
ground_df.head()

,question,id
0,Is almond fudge banana cake made with real ban...,142_1
1,Does almond fudge banana cake contain dairy?,142_1
2,Can I be sure there are no artificial flavors ...,142_1
3,Are the ingredients in almond fudge banana cak...,142_1
4,May I know the types of flour and sugar used i...,142_1


In [45]:
ground_truth = ground_df.to_dict(orient='records')


In [46]:
ground_truth[0]


{'question': 'Is almond fudge banana cake made with real bananas?',
 'id': '142_1'}

In [64]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [65]:
from tqdm.auto import tqdm 

In [66]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/16527 [00:00<?, ?it/s]

{'hit_rate': 0.5956918981061293, 'mrr': 0.3399384220722044}

In [23]:
import pandas as pd

def visualize_search_results(ground_truth, search_function):
    # Create an empty list to store the results
    search_results = []

    # Iterate over each query in the ground truth dataset
    for q in tqdm(ground_truth):
        doc_id = q['id']
        ground_truth_question = q['question']
        
        # Perform the search using the provided function
        results = search_function(q['question'])
        
        # Collect the top result from the search
        if results:
            matched_question = results[0]['question']
            matched_doc_id = results[0]['id']
        else:
            matched_question = None
            matched_doc_id = None

        # Append the comparison of the ground truth with the top matched result
        search_results.append({
            'Ground Truth ID': doc_id,
            'Ground Truth Question': ground_truth_question,
            'Matched Document ID': matched_doc_id,
            'Matched Question': matched_question
        })

    # Convert the results into a Pandas DataFrame for visualization
    results_df = pd.DataFrame(search_results)
    
    # Display the DataFrame
    pd.set_option('display.max_colwidth', None)
    return results_df

# Visualize the search results
results_df = visualize_search_results(ground_truth, lambda q: minsearch_search(q))
print(results_df.head())

# Optionally, you can also save this as an Excel or CSV file for further inspection
results_df.to_csv('search_results_comparison.csv', index=False)


  0%|          | 0/16527 [00:00<?, ?it/s]

  Ground Truth ID  \
0           142_1   
1           142_1   
2           142_1   
3           142_1   
4           142_1   

                                                        Ground Truth Question  \
0                         Is almond fudge banana cake made with real bananas?   
1                                Does almond fudge banana cake contain dairy?   
2  Can I be sure there are no artificial flavors in almond fudge banana cake?   
3         Are the ingredients in almond fudge banana cake free from liqueurs?   
4   May I know the types of flour and sugar used in almond fudge banana cake?   

  Matched Document ID                          Matched Question  
0              142_11  What tags are associated with this dish?  
1               142_3       How much total fat does it contain?  
2               142_5          How much sodium is in this dish?  
3               142_1                 What are the ingredients?  
4               142_4                What is the sugar c

In [24]:
ground_df.head(20)

,question,id
0,Is almond fudge banana cake made with real bananas?,142_1
1,Does almond fudge banana cake contain dairy?,142_1
2,Can I be sure there are no artificial flavors in almond fudge banana cake?,142_1
3,Are the ingredients in almond fudge banana cake free from liqueurs?,142_1
4,May I know the types of flour and sugar used in almond fudge banana cake?,142_1
5,How many calories does the Almond Fudge Banana Cake have?,142_2
6,Can I get nutrition facts for the Almond Fudge Banana Cake?,142_2
7,What's the calorie count for the Almond Fudge Banana Cake?,142_2
8,How many calories is the Almond Fudge Banana Cake?,142_2
9,What's the calorie content of the Almond Fudge Banana Cake?,142_2


In [25]:
documents[1:20]

[{'id': '142_2',
  'question': 'How many calories does it have?',
  'section': 'calories',
  'text': 'The almond fudge banana cake has 224.8 calories.',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_3',
  'question': 'How much total fat does it contain?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 14 grams of total fat (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_4',
  'question': 'What is the sugar content?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 87 grams of sugar (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_5',
  'question': 'How much sodium is in this dish?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 10 grams of sodium (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_6',
  'question': 'What is the protein content?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 7 grams of protein (PDV).',
  'dish_

## The RAG flow


In [26]:
def build_prompt(query, search_results):
    prompt_template = """
You're an AI assistant helping with menu queries. Answer the QUESTION based on the CONTEXT provided.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [27]:
client

In [28]:

def llm(prompt):
    response = client.chat.completions.create(
        model= MODEL_NAME,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [29]:
import minsearch

index = minsearch.Index(
    text_fields = ['id', 'question','section','text','dish_name'],
    keyword_fields=['dish_name']
    #keyword_fields=['id']
)


In [30]:
index.fit(documents)

In [3]:
import json
from time import time
from openai import OpenAI
import ingest
import json
import pandas as pd
from groq import Groq
from dotenv import load_dotenv
import os
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.auto import tqdm

# Setup the OpenAI client to use either Groq, OpenAI.com, or Ollama API
load_dotenv()
API_HOST = os.getenv("API_HOST")


if API_HOST == "groq":
    client = client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
    MODEL_NAME = os.getenv("GROQ_MODEL")

elif API_HOST == "ollama":
    client = openai.OpenAI(
        base_url=os.getenv("OLLAMA_ENDPOINT"),
        api_key="nokeyneeded",
    )
    MODEL_NAME = os.getenv("OLLAMA_MODEL")

elif API_HOST == "openai":
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    MODEL_NAME = os.getenv("OPENAI_MODEL")
else:
    print("No LLM Selected")


client = OpenAI()
index = ingest.load_index()


def minsearch_search(query):
    boost = {

    'question': 2.5,       # High priority: users are searching questions
    'dish_name': 1.7,      # Important for matching specific dishes
    'section': 1.2,        # Moderate boost: helps in filtering by category
    'text': 1.3        # Slight boost: could help match answer content}
    }

    results = index.search(
        query=query,
        filter_dict={'question': query},
        boost_dict=boost,
        num_results=10
    )

    return results


entry_template = """
question: {question}
dish_name: {dish_name}
section: {section}
text: {text}
""".strip()



prompt_template = """
You're an AI assistant helping with menu queries. Answer the QUESTION based on the CONTEXT from our database.
Use only the facts from the CONTEXT when answering the QUESTION.
QUESTION: {question}
CONTEXT: 
{context}
""".strip()


def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        #context += f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt



def llm(prompt, model=MODEL_NAME):
    response = client.chat.completions.create(
        model= MODEL_NAME,
        messages=[{"role": "user", "content": prompt}]
    )
    
    #return response.choices[0].message.content
    answer = response.choices[0].message.content

    token_stats = {
        "prompt_tokens": response.usage.prompt_tokens,
        "completion_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens,
    }

    return answer, token_stats

#def llm(prompt, model="gpt-4o-mini"):
    response = client.chat.completions.create(
        model=model, messages=[{"role": "user", "content": prompt}]
    )

    answer = response.choices[0].message.content

    token_stats = {
        "prompt_tokens": response.usage.prompt_tokens,
        "completion_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens,
    }

    return answer, token_stats


evaluation_prompt_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()


def evaluate_relevance(question, answer):
    prompt = evaluation_prompt_template.format(question=question, answer=answer)
    evaluation, tokens = llm(prompt, model=MODEL_NAME)

    try:
        json_eval = json.loads(evaluation)
        return json_eval, tokens
    except json.JSONDecodeError:
        result = {"Relevance": "UNKNOWN", "Explanation": "Failed to parse evaluation"}
        return result, tokens


def calculate_openai_cost(model, tokens):
    openai_cost = 0

    if model == MODEL_NAME:
        openai_cost = (
            tokens["prompt_tokens"] * 0.00015 + tokens["completion_tokens"] * 0.0006
        ) / 1000
    else:
        print("Model not recognized. OpenAI cost calculation failed.")

    return openai_cost


def rag(query, model=MODEL_NAME):
    t0 = time()

    search_results = minsearch_search(query)
    prompt = build_prompt(query, search_results)
    answer, token_stats = llm(prompt, model=model)

    relevance, rel_token_stats = evaluate_relevance(query, answer)

    t1 = time()
    took = t1 - t0

    openai_cost_rag = calculate_openai_cost(model, token_stats)
    openai_cost_eval = calculate_openai_cost(model, rel_token_stats)

    openai_cost = openai_cost_rag + openai_cost_eval

    answer_data = {
        "answer": answer,
        "model_used": model,
        "response_time": took,
        "relevance": relevance.get("Relevance", "UNKNOWN"),
        "relevance_explanation": relevance.get(
            "Explanation", "Failed to parse evaluation"
        ),
        "prompt_tokens": token_stats["prompt_tokens"],
        "completion_tokens": token_stats["completion_tokens"],
        "total_tokens": token_stats["total_tokens"],
        "eval_prompt_tokens": rel_token_stats["prompt_tokens"],
        "eval_completion_tokens": rel_token_stats["completion_tokens"],
        "eval_total_tokens": rel_token_stats["total_tokens"],
        "openai_cost": openai_cost,
    }
    print("sic")
    return answer_data

In [31]:
def minsearch(question):
    return index.search(question)

In [32]:
def rag(query):
    search_results = minsearch(query)
    prompt=build_prompt(query,search_results)
    answer = llm(prompt)
    return answer


In [33]:
query = "what are the calories of white chocolate cake is it good?"
query = "what are the ingredients in white chocolate cake?"

rag(query)

'Based on the provided context, the ingredients in the white chocolate cake are:\n\n* White chocolate\n* Butter\n* Sugar\n* Egg yolks\n* Vanilla\n* Cake flour\n* Baking powder\n* Salt\n* Buttermilk\n* Pecans\n* Flaked coconut\n* Egg whites\n* Flour\n* Milk\n\nLet me know if you need any further assistance!'

In [34]:
ground_truth[2]['question']

'Can I be sure there are no artificial flavors in almond fudge banana cake?'

In [35]:
rag(ground_truth[1]['question'])

'Based on the context, there is no specific mention of dairy in the provided sections (nutritional, category, calories, rating, preparation, price). Therefore, the answer to the question "Does almond fudge banana cake contain dairy?" is: "Not specified or listed." If you need to know if the cake contains dairy, you may want to check with the restaurant or bakery that serves it.'

## RAG Evaluation - LLM as a judge

In [38]:
record = ground_truth[0]
question =  record['question']
answer_llm = rag(question)
answer_llm

'Based on the provided CONTEXT, the answer to the QUESTION "Is almond fudge banana cake made with real bananas?" is:\n\nNot explicitly stated in the given context. However, the presence of the word "banana" in the name "almond fudge banana cake" suggests that bananas are likely an ingredient. Additionally, the presence of 11 grams of carbohydrates (PDV) and 87 grams of sugar (PDV) implies that bananas are likely used in the recipe. However, without direct confirmation, we can\'t be certain.'

In [39]:
prompt = prompt2_template.format(question=question,answer_llm=answer_llm)
prompt

'You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.\nYour task is to analyze the relevance of the generated answer to the given question.\nBased on the relevance of the generated answer, you will classify it\nas "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".\n\nHere is the data for evaluation:\n\nQuestion: Is almond fudge banana cake made with real bananas?\nGenerated Answer: Based on the provided CONTEXT, the answer to the QUESTION "Is almond fudge banana cake made with real bananas?" is:\n\nNot explicitly stated in the given context. However, the presence of the word "banana" in the name "almond fudge banana cake" suggests that bananas are likely an ingredient. Additionally, the presence of 11 grams of carbohydrates (PDV) and 87 grams of sugar (PDV) implies that bananas are likely used in the recipe. However, without direct confirmation, we can\'t be certain.\n\nPlease analyze the content and context of the generated answer in relation to the question\n

In [40]:
record = ground_truth[0]
record

{'question': 'Is almond fudge banana cake made with real bananas?',
 'id': '142_1'}

## RAG EVALUATION

In [46]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [47]:
len(ground_truth)


16527

In [48]:
record = ground_truth[0]


In [49]:
print(answer_llm)


Based on the context provided, the answer to the question "What's the amount of sugar in the almond fudge banana cake?" is:

87 grams of sugar (PDV).


In [50]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: Is the almond fudge banana cake high in sugar?
Generated Answer: Based on the context provided, the answer to the question "What's the amount of sugar in the almond fudge banana cake?" is:

87 grams of sugar (PDV).

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}


In [53]:
import json
len(ground_df)

16527

In [70]:
df_sample = ground_df.sample(n=100, random_state=1)

In [71]:
sample  = df_sample.to_dict(orient='records')

In [72]:
df_sample

,question,id
3881,Are the cod cakes browned on both sides,11252_10
15077,Can you tell me the preparation time for Coconut Shrimp Curry?,30898_9
9392,Should I double the funnel cake recipe?,19611_13
4398,Tells me the calorie total for Streusel Plum Cake,12105_2
15297,Milky Way Lover's Pound Cake prep time,31334_9
...,...,...
14559,Is the cheesecake cups low in carbs?,30082_8
12518,Can I get a step-by-step guide for caramel apple cake?,26737_10
4108,Easy Lemon Pound Cake preparation process,11763_10
13302,Can I get an estimate for the timing on the chocolate angel food cake?,28138_9


In [73]:
for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)

    # Log the evaluation to inspect if it's valid JSON
    print(f"Evaluation for question {question}: {evaluation}")

    try:
        # Try to parse the evaluation response as JSON
        evaluation = json.loads(evaluation)
    except json.JSONDecodeError as e:
        # Handle JSON decode errors gracefully
        print(f"JSONDecodeError for question {question}: {e}")
        evaluation = None  # You can decide what to do in case of error

    evaluations.append((record, answer_llm, evaluation))


  0%|          | 0/100 [00:00<?, ?it/s]

Evaluation for question Are the cod cakes browned on both sides: {
"Relevance": "RELEVANT",
"Explanation": "The generated answer provides a direct answer to the question 'Are the cod cakes browned on both sides?' by referencing the specific cooking instructions which include frying the patties for 4-5 minutes on each side. This provides a clear resolution to the question."
JSONDecodeError for question Are the cod cakes browned on both sides: Expecting ',' delimiter: line 3 column 284 (char 310)
Evaluation for question Can you tell me the preparation time for Coconut Shrimp Curry?: {
  "Relevance": "PARTLY_RELEVANT",
  "Explanation": "The generated answer acknowledges the question about preparation time for Coconut Shrimp Curry and provides an explanation for not being able to provide the answer, but also offers alternative solutions, indicating partial relevance to the question."
JSONDecodeError for question Can you tell me the preparation time for Coconut Shrimp Curry?: Expecting ',' 

InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}

In [59]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

TypeError: 'NoneType' object is not subscriptable

In [148]:
df_eval.relevance.value_counts(normalize=True)


relevance
RELEVANT           0.38
PARTLY_RELEVANT    0.36
NON_RELEVANT       0.26
Name: proportion, dtype: float64

In [150]:
df_eval.to_csv(input_data_folder+'rag-eval-gpt-3.5-turbo.csv', index=False)


In [152]:
df_eval[df_eval.relevance == 'RELEVANT']


,answer,id,question,relevance,explanation
0,"The cod cakes are fried in a skillet for 4-5 minutes on each side until they are golden brown, so yes, they are browned on both sides.",11252_10,Are the cod cakes browned on both sides,RELEVANT,The generated answer directly addresses the question by confirming that the cod cakes are browned on both sides after being fried in a skillet for 4-5 minutes. It provides a clear and relevant response to the question.
4,Section: prep_time\nQuestion: What is the prep time for Milky Way Lover's Pound Cake?\nAnswer: The prep time for Milky Way Lover's Pound Cake is 30 minutes.,31334_9,Milky Way Lover's Pound Cake prep time,RELEVANT,The generated answer directly addresses the question by providing the specific prep time of 30 minutes for Milky Way Lover's Pound Cake. It is fully relevant to the question asked.
7,"No, the rhubarb stir cake does not have a high carb count. It contains 17 grams of carbohydrates.",14457_8,Does the rhubarb stir cake have a high carb count?,RELEVANT,"The generated answer directly addresses the question by stating that the rhubarb stir cake does not have a high carb count. It also provides specific information about the amount of carbohydrates it contains (17 grams), which further supports its relevance to the question."
9,Section: nutritional\nQuestion: What is the fat content of the key lime cheesecake squares?\nAnswer: The key lime cheesecake squares has 6 grams of total fat (PDV).,18007_7,What's the fat content of the key lime cheesecake squares?,RELEVANT,"The generated answer directly addresses the fat content of the key lime cheesecake squares mentioned in the question, providing the specific amount of total fat (6 grams). This indicates a clear and accurate answer to the question asked."
11,"Instruction for baking the 1 Pan Fudge Cake:\n1. Preheat oven to 350 degrees\n2. In an ungreased 9 x 13 pan (yes ungreased), sift all dry ingredients\n3. Add the liquids and stir just until blended\n4. Bake for 25 minutes\n5. Frost with your favorite frosting, cake is extremely moist so care must be taken that you don't tear up the top of the cake.",29935_10,What's the instruction for baking the 1 Pan Fudge Cake?,RELEVANT,"The generated answer provides the step-by-step instructions for baking the 1 Pan Fudge Cake as requested in the question. It includes details such as preheating the oven, using an ungreased pan, sifting dry ingredients, adding liquids, baking time, and frosting the cake. The answer directly addresses the question and is relevant to the topic."
13,"Section: ingredients\nQuestion: Tell me the Million Dollar Pound Cake's composition?\nAnswer: The million dollar pound cake contains the following ingredients: butter, sugar, eggs, all-purpose flour, milk, almond extract, vanilla extract.",16489_1,Tell me the Million Dollar Pound Cake's composition?,RELEVANT,"The answer directly provides the composition of the Million Dollar Pound Cake, listing all the ingredients included."
14,"Yes, based on the nutritional information provided, carrot snack cake is likely a sweet dish as it contains 46 grams of sugar.",8847_14,Is carrot snack cake a sweet dish?,RELEVANT,The generated answer directly answers the question by confirming that carrot snack cake is indeed a sweet dish based on the high sugar content mentioned in the nutritional information provided. The answer is clear and directly relevant to the question.
18,"No, the Momma's Fair Funnel Cake is not high in salt as it only contains 16 grams of sodium.",916_5,Is the Momma's Fair Funnel Cake high in salt?,RELEVANT,The generated answer directly addresses the question by stating that Momma's Fair Funnel Cake is not high in salt and provides a specific amount of sodium it contains. This information is relevant as it directly addresses the salt content of the funnel cake.
21,The peach upside down pudding cake contains 413.0 calories.,19324_2,How many calories does the peach upside down pudding cake contain,RELEVANT,"The g

## Ingestion

In [3]:
# change path for base folder
base_folder = 'D:/Projects/AI-Restaurent-Chat-bot/'
input_data_folder = base_folder+'input_data/'

In [17]:
import pandas as pd
documents


[{'id': '142_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The almond fudge banana cake contains the following ingredients: ['dole banana', 'sugar', 'margarine', 'eggs', 'amaretto liqueur', 'vanilla extract', 'all-purpose flour', 'unsweetened cocoa powder', 'baking soda', 'salt', 'dole almond']",
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_2',
  'question': 'How many calories does it have?',
  'section': 'calories',
  'text': 'The almond fudge banana cake has 224.8 calories.',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_3',
  'question': 'How much total fat does it contain?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 14 grams of total fat (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_4',
  'question': 'What is the sugar content?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 87 grams of sugar (PDV).',
  'dish_name': 'almond fudge banana cake'},
